In [3]:
import cv2
import numpy as np
def enhance_image(image):
    image_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCR_CB)
    Y, Cr, Cb = cv2.split(image_YCrCb)
    Y = cv2.equalizeHist(Y)
    image_YCrCb = cv2.merge([Y, Cr, Cb])
    image = cv2.cvtColor(image_YCrCb, cv2.COLOR_YCR_CB2BGR)
    return image
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

In [5]:
import dlib
import cv2
face_detector = dlib.get_frontal_face_detector()
#face_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")
def scale_faces(face_rects, down_scale=1.5):
    faces = []
    for face in face_rects:
        scaled_face = dlib.rectangle(int(face.left() * down_scale),
                                    int(face.top() * down_scale),
                                    int(face.right() * down_scale),
                                    int(face.bottom() * down_scale))
        faces.append(scaled_face)
    return faces
def detect_faces(image, down_scale=1.5):
    image_scaled = cv2.resize(image, None, fx=1.0/down_scale, fy=1.0/down_scale, 
                              interpolation=cv2.INTER_LINEAR)
    faces = face_detector(image_scaled, 0)
  #  faces = [face.rect for face in faces]
    faces = scale_faces(faces, down_scale)
    return faces

In [ ]:
if __name__ == "__main__":
    image = cv2.imread("A.jpg")
    faces = detect_faces(image, down_scale=0.5)
    for face in faces:
        x,y,w,h = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(image, (x,y), (w,h), (255,200,150), 2, cv2.CV_AA)
    cv2.imshow("Image", image)
    cv2.waitKey(0)

In [ ]:
import numpy as np
def extract_face_embeddings(image, face_rect,shape_predictor,face_recognizer):
    shape = shape_predictor(image, face_rect)
    face_embedding = face_recognizer.compute_face_descriptor(image, shape)
    face_embedding = [x for x in face_embedding]
    face_embedding = np.array(face_embedding, dtype="float32")[np.newaxis, :]
    return face_embedding

In [ ]:
import numpy as np
import cPickle
def add_embeddings(embedding, label, 
                   embeddings_path="face_embeddings.npy", 
                   labels_path="labels.pickle"):
    first_time = False
    try:
        embeddings = np.load(embeddings_path)
        labels = cPickle.load(open(labels_path))
    except IOError:
        first_time = True
    if first_time:
        embeddings = embedding
        labels = [label]
    else:
        embeddings = np.concatenate([embeddings, embedding], axis=0)
        labels.append(label)
    np.save(embeddings_path, embeddings)
    with open(labels_path, "w") as f:
        cPickle.dump(labels, f)
    return True

In [ ]:
from extractors import extract_face_embeddings
from detectors import detect_faces
from db import add_embeddings
import dlib
shape_predictor = dlib.shape_predictor("models/shape_predictor_5_face_landmarks.dat")
face_recognizer = dlib.face_recognition_model_v1("models/dlib_face_recognition_resnet_model_v1.dat")
def enroll_face(image, label,
                embeddings_path="face_embeddings.npy",
                labels_path="labels.pickle", down_scale=1.0):
    faces = detect_faces(image, down_scale)
    if len(faces)<1:
        return False
    if len(faces)>1:
        raise ValueError("Multiple faces not allowed for enrolling")
    face = faces[0]
    face_embeddings = extract_face_embeddings(image, face, shape_predictor, 
                                              face_recognizer)
    add_embeddings(face_embeddings, label, embeddings_path=embeddings_path,
                   labels_path=labels_path)
    return True
if __name__ == "__main__":
    import cv2
    import glob
    import argparse
    ap = argparse.ArgumentParser()
    ap.add_argument("-d","--dataset", help="Path to dataset to enroll", required=True)
    ap.add_argument("-e","--embeddings", help="Path to save embeddings",
                    default="face_embeddings.npy")
    ap.add_argument("-l","--labels", help="Path to save labels",
                    default="labels.cpickle")
    args = vars(ap.parse_args())
    filetypes = ["png", "jpg"]
    dataset = args["dataset"].rstrip("/")
    imPaths = []
    for filetype in filetypes:
        imPaths += glob.glob("{}/*/*.{}".format(dataset, filetype))
    for path in imPaths:
        label = path.split("/")[-2]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        enroll_face(image, label, embeddings_path=args["embeddings"],
                    labels_path=args["labels"])

In [ ]:
import numpy as np
def recognize_face(embedding, embeddings, labels, threshold=0.5):
    distances = np.linalg.norm(embeddings - embedding, axis=1)
    argmin = np.argmin(distances)
    minDistance = distances[argmin]
    if minDistance>threshold:
        label = "Unknown"
    else:
        label = labels[argmin]
    return (label, minDistance)

In [ ]:
if __name__ == "__main__":
    import cv2
    import argparse
    from detectors import detect_faces
    from extractors import extract_face_embeddings
    import cPickle
    import dlib
    ap = argparse.ArgumentParser()
    ap.add_argument("-i","--image", help="Path to image", required=True)
    ap.add_argument("-e","--embeddings", help="Path to saved embeddings",
                    default="face_embeddings.npy")
    ap.add_argument("-l", "--labels", help="Path to saved labels",
                    default="labels.pickle")
    args = vars(ap.parse_args())
    embeddings = np.load(args["embeddings"])
    labels = cPickle.load(open(args["labels"]))
    shape_predictor = dlib.shape_predictor("models/"
                                           "shape_predictor_5_face_landmarks.dat")
    face_recognizer = dlib.face_recognition_model_v1("models/"
                                           "dlib_face_recognition_resnet_model_v1.dat")
    image = cv2.imread(args["image"])
    image_original = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = detect_faces(image)
    for face in faces:
        embedding = extract_face_embeddings(image, face, shape_predictor, face_recognizer)
        label = recognize_face(embedding, embeddings, labels)
        (x1, y1, x2, y2) = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(image_original, (x1, y1), (x2, y2), (255, 120, 120), 2, cv2.CV_AA)
        cv2.putText(image_original, label[0], (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
    cv2.imshow("Image", image_original)
    cv2.waitKey(0)

In [1]:
import minifinal.py

AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
import cv2
import numpy as np

hand = cv2.imread('Capture.png',0)

ret, the = cv2.threshold(hand, 70, 255, cv2.THRESH_BINARY)

_,contours,_ = cv2.findContours(the.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

hull = [cv2.convexHull(c) for c in contours]
final = cv2.drawContours(hand, hull, -1, (255,0,0))

cv2.imshow('Originals', hand)
cv2.imshow('Thresh',the)
cv2.imshow('Convex hull',final)

cv2.waitKey(0)
cv2.destroyAllWindows()